In [1]:
from wmyblog import *
os.environ['http_proxy'] = "http://127.0.0.1:7890" #代理的端口
os.environ['https_proxy'] = "http://127.0.0.1:7890"

In [4]:
import pandas as pd
def mergeComment(df_comment_new,commentFile='./data/comment_full.pkl',tag=''):

    df_comment = pd.read_pickle(commentFile)
    min_comment_date = 
    df_comment1 = df_comment.loc[df_comment.comment_date >= min(df_comment_new.comment_date)]
    df_comment2 = df_comment.loc[df_comment.comment_date < min(df_comment_new.comment_date)]

    id_list = df_comment.id.drop_duplicates()
    for id in id_list:
        d_new = df_comment_new.loc[df_comment_new.id == id]
        if not d_new.empty:
            d = df_comment1.loc[(df_comment1.id == id) & (df_comment1.comment_date >= min(d_new.comment_date))]
            if tag == 'full':
                idx0 = (df_comment1.id == id) & (df_comment1.comment_date < min(d_new.comment_date))
                df_comment1.loc[idx0,'comment'] = '<strike>'+df_comment1.loc[idx0,'comment']+'</strike>'
            d_merge = pd.merge(d_new,d,how='outer',on='comment',indicator=True)
            for idx in d_merge.loc[d_merge['_merge'] == 'right_only'].index:
                comment_date = d_merge.loc[idx,'comment_date_y']
                nickname = d_merge.loc[idx,'nickname_y']
                idx1 = (df_comment1.comment_date == comment_date) & (df_comment1.nickname == nickname)
                df_comment1.loc[idx1,'comment'] = '<strike>'+df_comment1.loc[idx1,'comment']+'</strike>'
            for idx in d_merge.loc[d_merge['_merge'] == 'both'].index:
                comment_date = d_merge.loc[idx,'comment_date_y']
                nickname = d_merge.loc[idx,'nickname_y']
                idx1 = (df_comment_new.comment_date == comment_date) & (df_comment_new.nickname == nickname)
                idx2 = (df_comment1.comment_date == comment_date) & (df_comment1.nickname == nickname)
                if ~ df_comment1['first_reply_date'].isnull()[idx2].iloc[0]:
                    df_comment_new.loc[idx1,'first_reply_date'] = df_comment1.loc[idx2,'first_reply_date'].iloc[0]
    df_comment_new = pd.concat([df_comment_new,df_comment1],ignore_index=True).drop_duplicates(subset=['nickname','comment_date'])
    df_comment_new = pd.concat([df_comment_new,df_comment2],ignore_index=True)
    df_comment_new.comment = df_comment_new.comment.str.replace('<strike><strike>','<strike>').str.replace('</strike></strike>','</strike>')
    df_comment_new = df_comment_new.sort_values('comment_date',ascending=False).reset_index(drop=True)
    return df_comment_new

In [20]:
df_comment_new = pd.read_pickle('comment.pkl')
#df_comment_new = mergeComment(df_comment_new)

In [44]:
min(df_comment_new.comment_date.dropna())

Timestamp('2017-10-31 04:28:00')

In [41]:
min(df_comment_new.comment_date[0:1434])

NaT

In [29]:
type(df_comment_new.comment_date[0])

pandas._libs.tslibs.nattype.NaTType